# Análise de homologias por BLAST

In [72]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW 
import requests

In [73]:
def blast(accession : str, program : str, database : str, filename : str):
    """
    função que executa um blast de uma sequência do NCBI e guarda o resultado
    recebe o identificador da seq no NCBI, o tipo de programa de blast, a base de dados usada 
    e o nome do ficheiro que queremos guardar a informação
    """
    result_handle = NCBIWWW.qblast(program, database, accession)

    save_file = open(filename, "w")
    save_file.write(result_handle.read())
    save_file.close()
    result_handle.close()

    result_handle = open(filename)

    blast_record = NCBIXML.read(result_handle)

    return blast_record

In [74]:
def blast_filter(blast_record, e_value_threshold : float, coverage_threshold : float, per_identity_threshold : float):
    filtered_alignments = []
    for alignment in blast_record.alignments:
        for hsp in alignment.hsps:
            # coverage = hsp.align_length / blast_record.query_length
            e_value = hsp.expect
            coverage = (hsp.query_end - hsp.query_start + 1) / blast_record.query_length * 100
            per_identity = (hsp.identities / hsp.align_length) * 100
            if e_value <= e_value_threshold and coverage >= coverage_threshold and per_identity >= per_identity_threshold:
                print('\n****Alignment****')
                print('acession:', alignment.accession)
                print('title:', alignment.title)
                print('alignment length:', alignment.length)
                print('e value:', hsp.expect)
                #print('hsp length:', hsp.align_length)
                #print('hsps:', len(alignment.hsps))
                filtered_alignments.append(alignment)
    return filtered_alignments

In [81]:
def extract_ncbi_id_from_alignment(alignment):
    """
    Extracts the UniProt ID from a Bio.Blast.Record.Alignment object.

    Parameters:
    - alignment: Bio.Blast.Record.Alignment object

    Returns:
    - UniProt ID (str) if found, else None
    """
    try:
        # Access the sequence identifier from the alignment object
        seq_id = alignment.hit_id

        # Extract the UniProt ID from the sequence identifier
        uniprot_id_start = seq_id.find("|") + 1
        uniprot_id_end = seq_id.find("|", uniprot_id_start)
        uniprot_id = seq_id[uniprot_id_start:uniprot_id_end]

        return uniprot_id

    except AttributeError:
        # Handle cases where the required attributes are not present
        return None

In [95]:
def get_protein_function(ncbi_id):
    # UniProt API endpoint para devolver informação sobre a proteína
    api_url = f'https://www.ebi.ac.uk/proteins/api/proteins/{ncbi_id}'

    try:
        # Fazer um GET request para o UniProt API
        response = requests.get(api_url)

        # Verificar se o pedido foi bem sucedido (status code 200)
        if response.status_code == 200:
            # Parse da resposta JSON
            protein_data = response.json()

            # Extrair a função da proteína
            if 'comments' in protein_data:
                for comment in protein_data['comments']:
                    if 'text' in comment and comment['type'] == 'FUNCTION':
                        return comment['text'][0]

            return "Informação sobre a função não disponível."

        else:
            # Imprimir uma mensagem de erro se o pedido não for bem sucedido
            return f"Erro: Incapaz de retribuir informação para {ncbi_id}. Status code: {response.status_code}"

    except requests.exceptions.RequestException as e:
        # Lidar com exceções do pedido
        return f"Error: {e}"

## Gene FLG

BLAST contra a base de dados swissprot

In [12]:
flg_swissprot_blast_record = blast("NP_002007.1", "blastp", "swissprot", "flg_swissprot_protein_blast")

In [13]:
print(len(flg_swissprot_blast_record.alignments), "hits")

22 hits


In [14]:
flg_swissprot_filtered_blast = blast_filter(flg_swissprot_blast_record, 0.05, 80, 80)
flg_swissprot_filtered_blast


****Alignment****
acession: P20930
title: sp|P20930.3| RecName: Full=Filaggrin [Homo sapiens]
alignment length: 4061
e value: 0.0


In [15]:
if len(flg_swissprot_filtered_blast) == 1:
    print(len(flg_swissprot_filtered_blast), "filtered sequence")
elif len(flg_swissprot_filtered_blast) == 0 or len(flg_swissprot_filtered_blast) >= 1:
    print(len(flg_swissprot_filtered_blast), "filtered sequences")

1 filtered sequence


BLAST contra a base de dados nr (non-redundant)

In [16]:
flg_nr_blast_record = blast("NP_002007.1", "blastp", "nr", "flg_nr_protein_blast")

In [17]:
print(len(flg_nr_blast_record.alignments), "hits")

50 hits


In [18]:
flg_nr_filtered_blast = blast_filter(flg_nr_blast_record, 0.05, 80, 80)
flg_nr_filtered_blast


****Alignment****
acession: NP_002007
title: ref|NP_002007.1| filaggrin [Homo sapiens] >sp|P20930.3| RecName: Full=Filaggrin [Homo sapiens] >gb|ACX32320.1| filaggrin [synthetic construct]
alignment length: 4061
e value: 0.0

****Alignment****
acession: AFH55059
title: gb|AFH55059.1| truncated profilaggrin [Homo sapiens]
alignment length: 4021
e value: 0.0

****Alignment****
acession: KAI4082684
title: gb|KAI4082684.1| filaggrin [Homo sapiens]
alignment length: 4061
e value: 0.0

****Alignment****
acession: KAI2519162
title: gb|KAI2519162.1| filaggrin [Homo sapiens]
alignment length: 4061
e value: 0.0

****Alignment****
acession: PNI23036
title: gb|PNI23036.1| FLG isoform 1 [Pan troglodytes]
alignment length: 4060
e value: 0.0

****Alignment****
acession: XP_024204293
title: ref|XP_024204293.2| filaggrin [Pan troglodytes]
alignment length: 4060
e value: 0.0

****Alignment****
acession: XP_030858012
title: ref|XP_030858012.2| filaggrin [Gorilla gorilla gorilla]
alignment length: 4060
e 

In [19]:
if len(flg_nr_filtered_blast) == 1:
    print(len(flg_nr_filtered_blast), "filtered sequence")
elif len(flg_nr_filtered_blast) == 0 or len(flg_nr_filtered_blast) >= 1:
    print(len(flg_nr_filtered_blast), "filtered sequences")

24 filtered sequences


**Não esquecer de ver a abrangencia taxonomica e os dominios conservados das seqs homologas e comparar com a query**

## Gene C11ORF30

BLAST contra a base de dados swissprot

In [22]:
c11orf30_swissprot_blast_record = blast("Q7Z589.2", "blastp", "swissprot", "c11orf30_swissprot_protein_blast")

In [23]:
print(len(c11orf30_swissprot_blast_record.alignments), "hits")

3 hits


In [24]:
c11orf30_swissprot_filtered_blast = blast_filter(c11orf30_swissprot_blast_record, 0.05, 80, 80)
c11orf30_swissprot_filtered_blast


****Alignment****
acession: Q7Z589
title: ref|NP_064578.2| BRCA2-interacting transcriptional repressor EMSY isoform 4 [Homo sapiens] >ref|XP_003254738.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Nomascus leucogenys] >ref|XP_016777137.1| BRCA2-interacting transcriptional repressor EMSY isoform X8 [Pan troglodytes] >ref|XP_034789296.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Pan paniscus] >ref|XP_055138967.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Symphalangus syndactylus] >sp|Q7Z589.2| RecName: Full=BRCA2-interacting transcriptional repressor EMSY [Homo sapiens] >gb|EAW74999.1| chromosome 11 open reading frame 30, isoform CRA_d [Homo sapiens] >gb|KAI2561948.1| EMSY transcriptional repressor, BRCA2 interacting [Homo sapiens] >gb|KAI2561949.1| EMSY transcriptional repressor, BRCA2 interacting [Homo sapiens] >gb|KAI4073267.1| EMSY transcriptional repressor, BRCA2 interacting [Homo sapiens] >gb|KAI4073274.1| EMSY transcriptiona

In [25]:
if len(c11orf30_swissprot_filtered_blast) == 1:
    print(len(c11orf30_swissprot_filtered_blast), "filtered sequence")
elif len(c11orf30_swissprot_filtered_blast) == 0 or len(c11orf30_swissprot_filtered_blast) >= 1:
    print(len(c11orf30_swissprot_filtered_blast), "filtered sequences")

2 filtered sequences


BLAST contra a base de dados nr (non-redundant)

In [26]:
c11orf30_nr_blast_record = blast("Q7Z589.2", "blastp", "nr", "c11orf30_nr_protein_blast")

In [27]:
print(len(c11orf30_nr_blast_record.alignments), "hits")

50 hits


In [34]:
c11orf30_nr_filtered_blast = blast_filter(c11orf30_nr_blast_record, 0.05, 99, 99)
c11orf30_nr_filtered_blast


****Alignment****
acession: XP_024111401
title: ref|XP_024111401.1| BRCA2-interacting transcriptional repressor EMSY isoform X9 [Pongo abelii] >ref|XP_054295825.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Pongo pygmaeus] >gb|PNJ51184.1| EMSY isoform 1 [Pongo abelii] >gb|PNJ51185.1| EMSY isoform 2 [Pongo abelii]
alignment length: 1322
e value: 0.0

****Alignment****
acession: NP_064578
title: ref|NP_064578.2| BRCA2-interacting transcriptional repressor EMSY isoform 4 [Homo sapiens] >ref|XP_003254738.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Nomascus leucogenys] >ref|XP_016777137.1| BRCA2-interacting transcriptional repressor EMSY isoform X8 [Pan troglodytes] >ref|XP_034789296.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Pan paniscus] >ref|XP_055138967.1| BRCA2-interacting transcriptional repressor EMSY isoform X5 [Symphalangus syndactylus] >sp|Q7Z589.2| RecName: Full=BRCA2-interacting transcriptional repressor EMSY [Homo sapi

In [35]:
if len(c11orf30_nr_filtered_blast) == 1:
    print(len(c11orf30_nr_filtered_blast), "filtered sequence")
elif len(c11orf30_nr_filtered_blast) == 0 or len(c11orf30_nr_filtered_blast) >= 1:
    print(len(c11orf30_nr_filtered_blast), "filtered sequences")

29 filtered sequences


## Gene TSLP

BLAST contra a base de dados swissprot

In [36]:
tslp_swissprot_blast_record = blast("AAK67490.1", "blastp", "swissprot", "tslp_swissprot_protein_blast")

In [37]:
print(len(tslp_swissprot_blast_record.alignments), "hits")

2 hits


In [38]:
tslp_swissprot_filtered_blast = blast_filter(tslp_swissprot_blast_record, 0.05, 80, 80)
tslp_swissprot_filtered_blast


****Alignment****
acession: Q969D9
title: sp|Q969D9.1| RecName: Full=Thymic stromal lymphopoietin; Flags: Precursor [Homo sapiens]
alignment length: 159
e value: 2.53914e-117


In [39]:
if len(tslp_swissprot_filtered_blast) == 1:
    print(len(tslp_swissprot_filtered_blast), "filtered sequence")
elif len(tslp_swissprot_filtered_blast) == 0 or len(tslp_swissprot_filtered_blast) >= 1:
    print(len(tslp_swissprot_filtered_blast), "filtered sequences")

1 filtered sequence


BLAST contra a base de dados nr (non-redundant)

In [40]:
tslp_nr_blast_record = blast("AAK67490.1", "blastp", "nr", "tslp_nr_protein_blast")

In [41]:
print(len(tslp_nr_blast_record.alignments), "hits")

50 hits


In [45]:
tslp_nr_filtered_blast = blast_filter(tslp_nr_blast_record, 0.05, 80, 80)
tslp_nr_filtered_blast


****Alignment****
acession: NP_149024
title: ref|NP_149024.1| thymic stromal lymphopoietin isoform 1 precursor [Homo sapiens] >ref|XP_057158304.1| thymic stromal lymphopoietin isoform X2 [Pan paniscus] >sp|Q969D9.1| RecName: Full=Thymic stromal lymphopoietin; Flags: Precursor [Homo sapiens] >gb|PNI34937.1| TSLP isoform 2 [Pan troglodytes] >dbj|BAJ20847.1| thymic stromal lymphopoietin, partial [synthetic construct] >gb|AAK60617.1| thymic stromal lymphopoietin protein TSLP [Homo sapiens] >gb|AAK67490.1| thymic stromal lymphopoietin [Homo sapiens] >gb|EAW49037.1| thymic stromal lymphopoietin, isoform CRA_a [Homo sapiens]
alignment length: 159
e value: 3.54175e-114

****Alignment****
acession: PNJ80532
title: gb|PNJ80532.1| TSLP isoform 3, partial [Pongo abelii]
alignment length: 159
e value: 5.97347e-111

****Alignment****
acession: 5J12_A
title: pdb|5J12|A Chain A, Thymic stromal lymphopoietin [Homo sapiens]
alignment length: 163
e value: 1.61083e-106

****Alignment****
acession: XP_016

In [46]:
if len(tslp_nr_filtered_blast) == 1:
    print(len(tslp_nr_filtered_blast), "filtered sequence")
elif len(tslp_nr_filtered_blast) == 0 or len(tslp_nr_filtered_blast) >= 1:
    print(len(tslp_nr_filtered_blast), "filtered sequences")

21 filtered sequences


## Web scrapping da função das sequências homólogas

In [96]:
ncbi_ids = []
for alignment in tslp_nr_filtered_blast:
    ncbi_id = extract_ncbi_id_from_alignment(alignment)
    ncbi_ids.append(ncbi_id)
print(ncbi_ids)

['NP_149024.1', 'PNJ80532.1', '5J12', 'XP_016809101.2', 'XP_047273802.1', 'XP_003826800.1', 'XP_054344267.1', 'XP_003259868.2', 'XP_011721784.1', 'XP_001100503.1', 'XP_025244335.1', 'XP_011895318.1', 'EHH26699.1', 'XP_031522246.1', 'XP_011844791.1', 'XP_050649065.1', 'XP_005557555.2', 'XP_005557555.1', '5J11', 'XP_003920983.1', 'XP_017382658.1']


In [97]:
for ncbi_id in ncbi_ids:
    protein_function = get_protein_function(ncbi_id)
    print(f"{ncbi_id} function: {protein_function}")

NP_149024.1 function: Erro: Incapaz de retribuir informação para NP_149024.1. Status code: 400
PNJ80532.1 function: Erro: Incapaz de retribuir informação para PNJ80532.1. Status code: 400
5J12 function: Erro: Incapaz de retribuir informação para 5J12. Status code: 400
XP_016809101.2 function: Erro: Incapaz de retribuir informação para XP_016809101.2. Status code: 400
XP_047273802.1 function: Erro: Incapaz de retribuir informação para XP_047273802.1. Status code: 400
XP_003826800.1 function: Erro: Incapaz de retribuir informação para XP_003826800.1. Status code: 400
XP_054344267.1 function: Erro: Incapaz de retribuir informação para XP_054344267.1. Status code: 400
XP_003259868.2 function: Erro: Incapaz de retribuir informação para XP_003259868.2. Status code: 400
XP_011721784.1 function: Erro: Incapaz de retribuir informação para XP_011721784.1. Status code: 400
XP_001100503.1 function: Erro: Incapaz de retribuir informação para XP_001100503.1. Status code: 400
XP_025244335.1 function:

In [99]:
get_protein_function("NP_149024.1 ")

'Erro: Incapaz de retribuir informação para NP_149024.1 . Status code: 400'